In [3]:
import pandas as pd
import numpy as np
import gudhi as gd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from matplotlib.animation import FuncAnimation
from sklearn.manifold import MDS
import gc  # 引入垃圾回收模块

In [4]:

# -------------------------- 1. 参数与数据读取（核心优化：控制行业数量） --------------------------
H2_PERSISTENCE_THRESHOLD = 0.00001
WINDOW_SIZE = 31
MAX_INDUSTRIES = 30  # 限制最大行业数量（关键优化：n=30时Rips复形内存可控，>50易爆炸）


# 读取数据并筛选关键字段
df_raw = pd.read_csv(
    r"datasets/ASWSIndexEOD.csv",
    skiprows=[1],
    usecols=["S_INFO_WINDCODE", "TRADE_DT", "S_DQ_CLOSE"],
    parse_dates=["TRADE_DT"],
    dtype={"S_DQ_CLOSE": "float64"}
)

# 长转宽（保留数据最完整的前MAX_INDUSTRIES个行业，减少点云规模）
industry_counts = df_raw["S_INFO_WINDCODE"].value_counts()  # 统计每个行业的非空数据量
top_industries = industry_counts.index[:MAX_INDUSTRIES].tolist()  # 取数据最完整的前30个行业
df_filtered = df_raw[df_raw["S_INFO_WINDCODE"].isin(top_industries)]  # 筛选行业

# 转宽格式（行=日期，列=行业，值=收盘价）
df = df_filtered.pivot_table(
    index="TRADE_DT",
    columns="S_INFO_WINDCODE",
    values="S_DQ_CLOSE",
    aggfunc="first"
).reset_index()

df.set_index("TRADE_DT", inplace=True)
df.fillna(method="ffill", inplace=True)  # 用前值填充缺失（比0填充更合理，减少噪声）
df.fillna(0, inplace=True)  # 仍有缺失则用0填充

print(f"优化后的数据规模：{df.shape}（日期数×行业数）")
if df.shape[1] < 3:
    raise ValueError("行业数量过少，无法进行拓扑分析（至少需要3个行业）")

优化后的数据规模：(6231, 30)（日期数×行业数）


C:\Users\59183\AppData\Local\Temp\ipykernel_32296\3295537557.py:30: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="ffill", inplace=True)  # 用前值填充缺失（比0填充更合理，减少噪声）


In [ ]:
import pandas as pd
iris = pd.read_csv('datasets/iris.csv', sep = ',')
iris

In [5]:

# -------------------------- 2. 动画生成函数（核心优化：降低复形维度+减少帧数） --------------------------
def generate_animation_for_window(points, max_edge_length, start_index, end_index):
    print(f"\n找到目标窗口 [{start_index}, {end_index})。开始生成动画...")
    
    # 构建Rips复形（关键优化：max_dimension=2，只保留到2维单纯形，无需3维）
    rips_complex = gd.RipsComplex(points=points, max_edge_length=max_edge_length)
    simplex_tree = rips_complex.create_simplex_tree(max_dimension=2)  # 原代码是3，此处降维
    simplex_tree.compute_persistence()
    
    # 设置画布
    fig = plt.figure(figsize=(18, 9))
    ax_complex = fig.add_subplot(1, 2, 1, projection='3d')
    ax_barcode = fig.add_subplot(1, 2, 2)

    # 绘制条形码
    persistence_pairs = simplex_tree.persistence()
    gd.plot_persistence_barcode(persistence=persistence_pairs, axes=ax_barcode, legend=True)
    ax_barcode.set_title(f"Barcode for Window [{start_index}, {end_index})")
    epsilon_line = ax_barcode.axvline(x=0, color='k', linestyle='--', linewidth=1.5)

    # 动画参数（关键优化：减少帧数，从150→100，降低内存占用）
    num_frames = 100  # 原150，减少33%
    epsilon_values = np.linspace(0, max_edge_length, num_frames)
    
    # 筛选filtration：只保留2维及以下单纯形（进一步减少数据量）
    filtration = [
        (simplex, f_val) for simplex, f_val in simplex_tree.get_filtration()
        if len(simplex) <= 2  # 原代码包含3维，此处过滤
    ]

    # 动画更新函数
    def update(i):
        current_epsilon = epsilon_values[i]
        ax_complex.clear()
        ax_complex.scatter(points[:, 0], points[:, 1], points[:, 2], c='b', marker='o', s=15)
        
        edges, triangles = [], []
        for simplex, filtration_value in filtration:
            if filtration_value > current_epsilon:
                break  # 提前终止，减少循环次数
            if len(simplex) == 2:
                edges.append(points[simplex])
            elif len(simplex) == 3:
                triangles.append(points[simplex])

        # 绘制边和三角形（控制绘制量）
        for edge in edges[:1000]:  # 限制最大边数（防止过多导致内存爆）
            ax_complex.plot(edge[:, 0], edge[:, 1], edge[:, 2], 'k-', linewidth=0.7)
        if triangles:
            # 限制三角形数量（取前500个）
            poly_collection = Poly3DCollection(triangles[:500], facecolors='r', linewidths=0, alpha=0.2)
            ax_complex.add_collection3d(poly_collection)

        ax_complex.set_title(f"Rips Complex, $\\epsilon = {current_epsilon:.2f}$")
        min_vals, max_vals = points.min(axis=0), points.max(axis=0)
        ax_complex.set_xlim(min_vals[0] - 0.1, max_vals[0] + 0.1)
        ax_complex.set_ylim(min_vals[1] - 0.1, max_vals[1] + 0.1)
        ax_complex.set_zlim(min_vals[2] - 0.1, max_vals[2] + 0.1)

        epsilon_line.set_xdata([current_epsilon, current_epsilon])
        return fig,

    # 创建并保存动画
    ani = FuncAnimation(fig, update, frames=num_frames, interval=100, blit=False)
    output_filename = f"datasets/animation_window_{start_index}_to_{end_index-1}.gif"
    print(f"正在保存动画到 {output_filename}...")
    ani.save(output_filename, writer='pillow', fps=10)
    print("动画制作完成！")
    
    # 强制释放内存（关键优化）
    plt.close(fig)
    del ani, fig, ax_complex, ax_barcode, filtration, rips_complex, simplex_tree
    gc.collect()  # 主动触发垃圾回收


# -------------------------- 3. 滑动窗口主循环（核心优化：控制复形复杂度+内存回收） --------------------------
max_start_index = len(df) - WINDOW_SIZE
for i in range(max_start_index + 1):
    start_index = i
    end_index = i + WINDOW_SIZE
    
    print(f"\n--- 正在检验窗口:索引 {start_index} 到 {end_index-1} ---")

    # 提取窗口数据
    window_df = df.iloc[start_index:end_index]
    if len(window_df) < WINDOW_SIZE:
        print("窗口数据不完整，跳过")
        continue

    # 计算相关系数矩阵和距离矩阵
    corr_matrix = window_df.corr().fillna(0)
    distance_matrix_values = np.sqrt(np.clip(2 * (1 - corr_matrix.values), a_min=0.0, a_max=4.0))

    # 计算max_edge_length（关键优化：降低分位数，从30%→20%，减少边数量）
    upper_triangle_indices = np.triu_indices_from(distance_matrix_values, k=1)
    if len(upper_triangle_indices[0]) == 0:
        continue
    distances_off_diagonal = distance_matrix_values[upper_triangle_indices]
    max_edge_length = np.percentile(distances_off_diagonal, 20)  # 原30%，减少边数量

    # MDS降维（保持3D）
    points = MDS(
        n_components=3,
        dissimilarity='precomputed',
        random_state=42,
        normalized_stress=False
    ).fit_transform(distance_matrix_values)

    # 检查点云规模（防止意外）
    n_points = points.shape[0]
    print(f"当前窗口点云数量：{n_points}（控制在{MAX_INDUSTRIES}以内）")
    if n_points > MAX_INDUSTRIES * 1.2:  # 允许120%的浮动
        print(f"点云数量过多（{n_points}），跳过该窗口")
        del window_df, corr_matrix, distance_matrix_values, points  # 释放内存
        gc.collect()
        continue

    # 构建Rips复形并计算持续同调（复用动画函数的优化：max_dimension=2）
    rips_complex_check = gd.RipsComplex(points=points, max_edge_length=max_edge_length)
    simplex_tree_check = rips_complex_check.create_simplex_tree(max_dimension=2)  # 降维
    simplex_tree_check.compute_persistence()
    
    # 检查二维空腔
    all_persistence = simplex_tree_check.persistence()
    has_significant_hole = False
    for dim, (birth, death) in all_persistence:
        if dim == 2:
            if death == float('inf') or (death - birth) > H2_PERSISTENCE_THRESHOLD:
                has_significant_hole = True
                break

    # 生成动画或继续
    if not has_significant_hole:
        print(f"*** 成功！窗口 [{start_index}, {end_index-1}] 未发现显著二维空腔。***")
        generate_animation_for_window(points, max_edge_length, start_index, end_index)
        print("\n任务完成，程序已停止。")
        break
    else:
        print(f"窗口 [{start_index}, {end_index-1}] 存在显著空腔。继续下一个窗口...")

    # 强制释放当前窗口的内存（关键优化）
    del window_df, corr_matrix, distance_matrix_values, points, rips_complex_check, simplex_tree_check
    gc.collect()

else:
    print("\n扫描完所有窗口，未找到任何没有显著二维空腔的区间。")


--- 正在检验窗口:索引 0 到 30 ---
当前窗口点云数量：30（控制在30以内）
*** 成功！窗口 [0, 30] 未发现显著二维空腔。***

找到目标窗口 [0, 31)。开始生成动画...


d:\Anaconda3\Lib\site-packages\gudhi\persistence_graphical_tools.py:139: UserWarning: usetex mode requires ghostscript.
  warnings.warn("usetex mode requires ghostscript.")


正在保存动画到 datasets/animation_window_0_to_30.gif...
动画制作完成！

任务完成，程序已停止。
